<a href="https://colab.research.google.com/github/yarayear2056-cmd/Week4_pro/blob/main/Bird_Species.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kagglehub

In [3]:
import kagglehub
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torchvision.models import ResNet18_Weights
from torchvision import models
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import copy

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [5]:
path = kagglehub.dataset_download("pavangawande/indian-bird-species-dataset-traintest-split")
print("Path to dataset files:", path)

Using Colab cache for faster access to the 'indian-bird-species-dataset-traintest-split' dataset.
Path to dataset files: /kaggle/input/indian-bird-species-dataset-traintest-split


In [6]:
print(os.listdir(path))


['Birds25_Split']


In [7]:
base_path = os.path.join(path, os.listdir(path)[0])
print(os.listdir(base_path))

['test', 'train']


In [8]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

In [9]:
train_path = os.path.join(base_path,"train")
test_path = os.path.join(base_path,"test")

In [10]:
#dataset
train_data = datasets.ImageFolder(train_path, transform=transform)
test_data = datasets.ImageFolder(test_path, transform=transform)
#data loader
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [11]:
#for small data
small_train_data = Subset(train_data, range(1000))
small_test_data = Subset(test_data, range(500))

small_train_loader = DataLoader(small_train_data, batch_size=32, shuffle=True)
small_test_loader = DataLoader(small_test_data, batch_size=32)

In [12]:
#load model
num_classes = len(train_data.classes)

model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 145MB/s]


In [13]:
def get_model(freeze_backbone=True):
    model = models.resnet18(weights=ResNet18_Weights.DEFAULT)

    if freeze_backbone:
        for p in model.parameters():
            p.requires_grad = False

    model.fc = nn.Linear(model.fc.in_features, num_classes)
    model.to(device)
    return model


In [14]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    loss_sum, correct, total = 0,0,0

    for x,y in loader:
        x,y = x.to(device), y.to(device)

        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out,y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        pred = out.argmax(1)
        correct += (pred==y).sum().item()
        total += y.size(0)

    return loss_sum/len(loader), correct/total


def eval_one_epoch(model, loader, criterion):
    model.eval()
    loss_sum, correct, total = 0,0,0

    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            out = model(x)
            loss = criterion(out,y)

            loss_sum += loss.item()
            pred = out.argmax(1)
            correct += (pred==y).sum().item()
            total += y.size(0)

    return loss_sum/len(loader), correct/total

In [15]:
def run_research_experiment(name, model, train_loader, val_loader, epochs=3):
    print(f"\n===== Experiment: {name} =====")

    params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.SGD(params, lr=0.001, momentum=0.9)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    criterion = nn.CrossEntropyLoss()


    since = time.time()
    best_acc = 0.0
    best_model_wts = copy.deepcopy(model.state_dict())

    history = {"train_loss":[], "train_acc":[], "val_loss":[], "val_acc":[]}

    for epoch in range(epochs):
        print(f"Epoch {epoch}/{epochs-1}")
        print("-"*10)

        train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_acc = eval_one_epoch(model, val_loader, criterion)

        history["train_loss"].append(train_loss)
        history["train_acc"].append(train_acc)
        history["val_loss"].append(val_loss)
        history["val_acc"].append(val_acc)

        print(f"train Loss: {train_loss:.4f} Acc: {train_acc:.4f}")
        print(f"val   Loss: {val_loss:.4f} Acc: {val_acc:.4f}\n")


        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f"Training complete in {int(time_elapsed//60)}m {int(time_elapsed%60)}s")
    print(f"Best val Acc: {best_acc:.6f}")

    model.load_state_dict(best_model_wts)

    return


In [ ]:
freeze_small = get_model(freeze_backbone=True)

run_research_experiment(
    "freeze + small data",
    freeze_small,
    small_train_loader,
    small_test_loader

)


===== Experiment: freeze + small data =====
Epoch 0/2
----------
train Loss: 0.2998 Acc: 0.9470
val   Loss: 7.1594 Acc: 0.5960

Epoch 1/2
----------


In [ ]:
freeze_large = get_model(freeze_backbone=True)
run_research_experiment(
    "freeze + large data",
    freeze_large,
    train_loader,
    test_loader
)

In [ ]:
unfreeze_small = get_model(freeze_backbone=False)
run_research_experiment(
    "unfreeze + small data",
    unfreeze_small,
    small_train_loader,
    small_test_loader
)

In [ ]:
unfreeze_large = get_model(freeze_backbone=False)
run_research_experiment(
    "unfreeze + large data",
    unfreeze_large,
    train_loader,
    test_loader
)

In [ ]:
plt.figure(figsize=(10,6))

plt.plot(freeze_small["val_acc"], label="Freeze Small")
plt.plot(freeze_large["val_acc"], label="Freeze Large")
plt.plot(unfreeze_small["val_acc"], label="Unfreeze Small")
plt.plot(unfreeze_large["val_acc"], label="Unfreeze Large")

plt.xlabel("Epoch")
plt.ylabel("Validation Accuracy")
plt.title("Transfer Learning Comparison")
plt.legend()
plt.show()

In [ ]:
def show_images(loader, class_names, n=6):
    images, labels = next(iter(loader))

    plt.figure(figsize=(12,6))
    for i in range(n):
        plt.subplot(2, 3, i+1)
        img = images[i].permute(1,2,0)
        plt.imshow(img)
        plt.title(class_names[labels[i]])
        plt.axis("off")
    plt.show()

In [ ]:
show_images(train_loader, class_names)

In [ ]:
def plot_history(history, title):
    plt.figure(figsize=(10,5))

    plt.plot(history["train_acc"], label="Train Acc")
    plt.plot(history["val_acc"], label="Val Acc")

    plt.title(title)
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
plot_history(freeze_small, "Freeze + Small Data")

In [ ]:
def visualize_predictions(model, loader, class_names):
    model.eval()
    images, labels = next(iter(loader))
    images = images.to(device)

    with torch.no_grad():
        outputs = model(images)
        preds = torch.argmax(outputs, 1)

    plt.figure(figsize=(12,6))
    for i in range(6):
        plt.subplot(2,3,i+1)
        img = images[i].cpu().permute(1,2,0)
        plt.imshow(img)
        plt.title(f"Pred: {class_names[preds[i]]}")
        plt.axis("off")
    plt.show()

In [ ]:
visualize_predictions(best_model, test_loader, class_names)